# 匯入必要程式庫

匯入pandas、numpy、matplotlib等程式庫，用於資料處理、數學運算和視覺化。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta

# 載入市場資料

使用yfinance從Yahoo Finance載入黃金ETF (GLD)的歷史價格資料。

In [ ]:
# 載入黃金資料
end_date = datetime.now()
start_date = end_date - timedelta(days=365*5)  # 5年資料
gold_data = yf.download('GLD', start=start_date, end=end_date)
gold_prices = gold_data['Close']
print(gold_prices.head())

# 定義交易策略

定義一個簡單的移動平均交叉策略，包括短期和長期移動平均線。

In [ ]:
# 定義SMA交叉策略
def sma_crossover_strategy(data, short_window=50, long_window=200):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data
    signals['short_mavg'] = data.rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = data.rolling(window=long_window, min_periods=1).mean()
    signals['signal'] = 0.0
    signals.loc[short_window:, 'signal'] = np.where(signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0)
    signals['positions'] = signals['signal'].diff()
    return signals

# 實作策略邏輯

根據移動平均交叉產生買入和賣出訊號，並計算收益。

In [ ]:
# 應用策略
signals = sma_crossover_strategy(gold_prices)
print(signals.tail())

# 回測策略

使用歷史資料回測策略，計算總收益、勝率和最大回撤。

In [ ]:
# 回測函數
def backtest_strategy(signals, initial_capital=10000):
    positions = pd.DataFrame(index=signals.index).fillna(0.0)
    positions['GLD'] = 100 * signals['signal']  # 買100股
    portfolio = positions.multiply(signals['price'], axis=0)
    pos_diff = positions.diff()
    portfolio['holdings'] = (positions.multiply(signals['price'], axis=0)).sum(axis=1)
    portfolio['cash'] = initial_capital - (pos_diff.multiply(signals['price'], axis=0)).sum(axis=1).cumsum()
    portfolio['total'] = portfolio['cash'] + portfolio['holdings']
    portfolio['returns'] = portfolio['total'].pct_change()
    return portfolio

portfolio = backtest_strategy(signals)
total_return = (portfolio['total'].iloc[-1] - 10000) / 10000 * 100
print(f"總收益: {total_return:.2f}%")

# 視覺化結果

使用matplotlib繪製價格圖表、訊號點和累積收益曲線。

In [ ]:
# 視覺化
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
ax[0].plot(signals['price'], label='GLD Price')
ax[0].plot(signals['short_mavg'], label='50-day SMA')
ax[0].plot(signals['long_mavg'], label='200-day SMA')
ax[0].set_title('Gold Price and Moving Averages')
ax[0].legend()

ax[1].plot(portfolio['total'], label='Portfolio Value')
ax[1].set_title('Portfolio Value Over Time')
ax[1].legend()

plt.tight_layout()
plt.show()

# 執行策略

在VS Code的Jupyter notebook中執行策略，並輸出結果到輸出窗格。

In [ ]:
# 執行所有細胞來運行策略
print("策略執行完成。檢查圖表和總收益。")